# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| export
import pandas as pd
import wandb
import os
from fastcore.script import call_parse, Param
from datetime import datetime
from rich.progress import Progress

In [ ]:
#|export

@call_parse
def fetch_runs(
    entity: Param("Entity for the WANDB runs. If not provided, attempts to fetch from environment variable 'WANDB_ENTITY'.", str) = None,
    project: Param("Project for the WANDB runs. If not provided, attempts to fetch from environment variable 'WANDB_PROJECT'.", str) = None,
    output_file: Param("Path to save the output CSV. Default name format: {entity}-{project}-{mmddyy}.csv", str) = None
):
        
    api = wandb.Api()

    # Retrieve entity and project from environment variables if not provided
    if not entity: 
        entity = os.environ.get("WANDB_ENTITY")
        if not entity:
            raise ValueError("Entity not provided and environment variable 'WANDB_ENTITY' is not set.")
        print(f"Fetched entity from environment variable 'WANDB_ENTITY': {entity}.")
    
    if not project: 
        project = os.environ.get("WANDB_PROJECT")
        if not project:
            raise ValueError("Project not provided and environment variable 'WANDB_PROJECT' is not set.")
        print(f"Fetched project from environment variable 'WANDB_PROJECT': {project}.")

    # Validate entity and project while estimating total runs
    try:
        runs_sample = api.runs(path=f"{entity}/{project}", per_page=1)
        total_runs = len(runs_sample)
    except Exception as e:
        raise ValueError(f"Invalid entity '{entity}' or project '{project}': {str(e)}\n\nAlso, make sure you are properly authenticated. You can authenticate by using 'wandb.login() or setting the environment variable 'WANDB_API_KEY'")

    # Default CSV file name format
    if not output_file:
        date_str = datetime.now().strftime("%m%d%y")
        output_file = f"{entity}-{project}-{date_str}.csv"

    all_runs_data = []

    with Progress() as progress:
        task = progress.add_task("[cyan]Fetching runs...", total=total_runs)

        last_created_at = None
        while not progress.finished:
            filters = {}
            if last_created_at:
                filters["created_at"] = {"$gt": last_created_at}

            runs = api.runs(path=f"{entity}/{project}", per_page=50, order="created_at", filters=filters)
            for run in runs:
                run_data = {
                    "id": run.id,
                    "name": run.name,
                    **run.summary._json_dict,
                    **{f"config_{k}": v for k, v in run.config.items() if not k.startswith('_')}
                }
                all_runs_data.append(run_data)

            progress.update(task, advance=len(runs))
            if len(runs) > 0:
                last_created_at = runs[-1].created_at

    df = pd.DataFrame(all_runs_data)
    df.to_csv(output_file, index=False)
    print(f"Saved {len(df)} runs to {output_file}")